In [1]:
# Get emojis
with open('emoji256-sortedPop.txt', 'r', encoding='utf-8') as f:
    emoji_lines = f.read().splitlines()

emojis = []
for line in emoji_lines:
    emo = line.split()
    emojis.extend(emo)

print(emojis)

In [47]:
# Get codepoints as a number.
codepoints = list(map(ord, emojis))
print([format(x, '08x') for x in codepoints])

In [48]:
# Reduce c. Every emoji that passes the filter starts with 0x0001F,
# so remove first two bytes.
c = list(map(lambda x: x.to_bytes(4)[2:], codepoints))
c.sort()
for i, x in enumerate(c):
    print(str(format(i,'03d')) + ' ' + format(x[0], '02x') + format(x[1], '02x'))

In [45]:
from itertools import groupby

# Key is the entry in the lookup table. We want distribution for more control.
# An entry that only has 1 emoji pointing to it is very good. We can use x^y^x=y
# to set the hash of the emoji arbitrarily.
groups = []
for k, g in groupby(d, lambda x: x[0]):
    groups.append((k, [x[1] for x in g]))

# The tricky part is the entries with multiple emojis pointing to it. We need to
# come up with a lookup entry f.e. multigroup s.t. all emojis will hash to different
# values. This is like a jigsaw puzzle with all the multigroups except also the
# jigsaw pieces change depending on where you put it.
groups = [(k,v) for k,v in groups if len(v) > 1]

print(len(groups))
groups2 = [(k, [v[0] ^ xx ^ 128 for xx in v]) for k, v in groups]
print(sorted(groups2, key=lambda x: len(x[1])))

In [23]:
# Modification of Pearson hashing: for this problem, the first 2 bytes in the
# message are the exact same, so they are ignored.
# For the other bytes, message[3] has a better distribution than message[2],
# which has a repetitive first 4 bits. So if we reduce table lookups to once,
# then we should use message[3] as the key into the table.
def hash8(message: str, table) -> int:
    return table[message[1]] ^ message[0]

hashed = list(map(lambda x: hash8(x, table), c))
print(sorted(hashed))

In [24]:
# Brute force - try to find a table that works
from random import shuffle

def has_collision(sorted_array):
    for i in range(1, len(sorted_array)):
        if sorted_array[i] == sorted_array[i-1]:
            return True
    return False

# table = list(map(lambda x: format(x & 0xFFFF, "04x"), codepoints))
table = list(range(256))
shuffle(table)

tries = 0
while(has_collision(sorted(hashed))):
    shuffle(table)
    hashed = list(map(lambda x: hash8(x, table), c))
    tries += 1
#     print(tries, end='\r')